In [0]:
# Installing libaries 
!pip install -q kaggle

#importing req lib
import pandas as pd
import numpy as np
import datetime
from sklearn.ensemble import RandomForestRegressor
from google.colab import files
##################################
# preprocessor
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Models from Scikit-Learn
from sklearn.ensemble import RandomForestRegressor

# Model Evaluations
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error,mean_squared_log_error,mean_absolute_error,make_scorer
#Pipeline
from sklearn.pipeline import Pipeline
from datetime import datetime

In [3]:
#uploading kaggle credential json
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [4]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/kaggle.json
!kaggle competitions download -c bluebook-for-bulldozers 

 54% 5.00M/9.28M [00:00<00:00, 10.4MB/s]
100% 9.28M/9.28M [00:00<00:00, 17.2MB/s]
100% 5.00M/5.01M [00:00<00:00, 10.5MB/s]
100% 5.01M/5.01M [00:00<00:00, 10.2MB/s]
  0% 0.00/338k [00:00<?, ?B/s]
100% 338k/338k [00:00<00:00, 48.0MB/s]
 73% 5.00M/6.85M [00:00<00:00, 12.5MB/s]
100% 6.85M/6.85M [00:00<00:00, 15.5MB/s]
 53% 5.00M/9.38M [00:00<00:00, 9.99MB/s]
100% 9.38M/9.38M [00:00<00:00, 17.4MB/s]
  0% 0.00/297k [00:00<?, ?B/s]
100% 297k/297k [00:00<00:00, 96.0MB/s]
  0% 0.00/207k [00:00<?, ?B/s]
100% 207k/207k [00:00<00:00, 67.9MB/s]
  0% 0.00/192k [00:00<?, ?B/s]
100% 192k/192k [00:00<00:00, 62.0MB/s]
 71% 5.00M/7.06M [00:00<00:00, 12.6MB/s]
100% 7.06M/7.06M [00:00<00:00, 15.9MB/s]
  0% 0.00/10.8k [00:00<?, ?B/s]
100% 10.8k/10.8k [00:00<00:00, 11.0MB/s]
 94% 9.00M/9.59M [00:00<00:00, 12.5MB/s]
100% 9.59M/9.59M [00:00<00:00, 14.0MB/s]
  0% 0.00/291k [00:00<?, ?B/s]
100% 291k/291k [00:00<00:00, 96.6MB/s]
  0% 0.00/209k [00:00<?, ?B/s]
100% 209k/209k [00:00<00:00, 65.3MB/s]
  0% 0.00/316k 

In [8]:
!unzip TrainAndValid.zip
!unzip Test.csv.zip	


Archive:  TrainAndValid.zip
  inflating: TrainAndValid.csv       
Archive:  Test.csv.zip
replace Test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Test.csv                


In [0]:
# combined dataset of training and validation set
df = pd.read_csv("TrainAndValid.csv",parse_dates=['saledate'],low_memory=False) 
# test set
test_df = pd.read_csv("Test.csv",parse_dates=['saledate'],low_memory=False)
# sorting df according to the saledate
df.sort_values(by='saledate',inplace=True)

In [0]:
df.head().T 

In [0]:
df.info() # most of the features are having object DataType

In [0]:
test_df.info()

In [0]:
df.isna().sum()

In [0]:
test_df.isna().sum()

In [0]:
# Concatinatng all data 
# test_df has no SalePrice column , so its data points will have NaN in its SalePrice column when cancatenated with df 
Concat = pd.concat((df,test_df),axis = 0).reset_index(drop=True)

In [0]:
# Converting all columns with object dtype to category dtype
for label,content in Concat.items() :
    if pd.api.types.is_object_dtype(content):
        Concat[label] = content.astype('category')
        
# Enriching features
Concat['year'] = Concat.saledate.dt.year
Concat['month']= Concat.saledate.dt.month
Concat['day']= Concat.saledate.dt.day

In [0]:
cat=[] # list for storing all columns with 'cstegory' dtype
cat_missing = [] # list for storing columns with 'category' dtype and having missing values
num_missing = [] # list for storing columns with 'numerical' dtype and having missing values

for label,content in Concat.items():
    
    if pd.api.types.is_numeric_dtype(content): # checking for numerical features
        if content.isna().sum() > 0: # checking if the feature has any missing values
            Concat[f'{label}_ismissing'] = content.isna()
            num_missing.append(label)
            
    if pd.api.types.is_categorical_dtype(content): # checking for categorical features
        cat.append(label) 
        if content.isna().sum() > 0: # checking if the feature has any missing values
            Concat[f'{label}_ismissing'] = content.isna()
            cat_missing.append(label)
            
cat_not_missing = list(set(cat) - set(cat_missing))

In [0]:
# For missing values in categorical datatype, by default `-1` is assigned for its code, so adding 1 before creating new column
Concat[cat_missing] = Concat[cat_missing].apply(lambda i : i.cat.codes+1)

# For features with no missing values, simply assigning code
Concat[cat_not_missing] = Concat[cat_not_missing].apply(lambda i : i.cat.codes)

In [14]:
(Concat.isna().sum() !=0 ).sum() # out which one is SalePrice , which will not be considered

3

In [0]:
train_df = Concat.loc[Concat.saledate.dt.year < 2012, :].drop('saledate', axis=1)

valid_df = Concat.loc[Concat.saledate <= pd.Timestamp(
    year=2012, month=4, day=30)].loc[Concat.saledate >= pd.Timestamp(year=2012, month=1, day=1)].drop('saledate', axis=1)

test_df = Concat.loc[Concat.saledate >=
                     pd.Timestamp(year=2012, month=4, day=30), :].drop(['SalePrice','saledate'], axis=1)

In [0]:
train_df.shape

In [0]:
valid_df.shape

In [0]:
test_df.shape

In [0]:
train_df[num_missing].isna().sum()

In [0]:
valid_df[num_missing].isna().sum()

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
num_imputer = SimpleImputer(strategy='median')
transformer = ColumnTransformer(transformers=[('num_missing',num_imputer,train_df.columns)],remainder='passthrough',)

train_df_filled = transformer.fit_transform(train_df) # fitting on training data 
valid_df_filled = transformer.transform(valid_df) # transforming test based on training data to avoid data leakage

train_df_filled = pd.DataFrame(train_df_filled,columns=train_df.columns)
valid_df_filled = pd.DataFrame(valid_df_filled,columns=valid_df.columns)

In [17]:
train_df_filled

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,...,auctioneerID_ismissing,MachineHoursCurrentMeter_ismissing,UsageBand_ismissing,fiSecondaryDesc_ismissing,fiModelSeries_ismissing,fiModelDescriptor_ismissing,ProductSize_ismissing,Drive_System_ismissing,Enclosure_ismissing,Forks_ismissing,Pad_Type_ismissing,Ride_Control_ismissing,Stick_ismissing,Transmission_ismissing,Turbocharged_ismissing,Blade_Extension_ismissing,Blade_Width_ismissing,Enclosure_Type_ismissing,Engine_Horsepower_ismissing,Hydraulics_ismissing,Pushblock_ismissing,Ripper_ismissing,Scarifier_ismissing,Tip_Control_ismissing,Tire_Size_ismissing,Coupler_ismissing,Coupler_System_ismissing,Grouser_Tracks_ismissing,Hydraulics_Flow_ismissing,Track_Type_ismissing,Undercarriage_Pad_Width_ismissing,Stick_Length_ismissing,Thumb_ismissing,Pattern_Changer_ismissing,Grouser_Type_ismissing,Backhoe_Mounting_ismissing,Blade_Type_ismissing,Travel_Controls_ismissing,Differential_Type_ismissing,Steering_Controls_ismissing
0,1646770.0,9500.0,1126363.0,8434.0,132.0,18.0,1974.0,0.0,0.0,4659.0,1760.0,0.0,0.0,0.0,4.0,46.0,43.0,4.0,4.0,0.0,6.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
1,1821514.0,14000.0,1194089.0,10150.0,132.0,99.0,1980.0,0.0,0.0,1846.0,560.0,0.0,0.0,0.0,0.0,59.0,8.0,5.0,5.0,0.0,6.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,17.0,3.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
2,1505138.0,50000.0,1473654.0,4139.0,132.0,99.0,1978.0,0.0,0.0,2380.0,714.0,62.0,0.0,0.0,2.0,49.0,8.0,4.0,4.0,0.0,6.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
3,1671174.0,16000.0,1327630.0,8591.0,132.0,99.0,1980.0,0.0,0.0,1845.0,559.0,0.0,0.0,0.0,0.0,73.0,8.0,5.0,5.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,17.0,3.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
4,1329056.0,22000.0,1336053.0,4089.0,132.0,99.0,1984.0,0.0,0.0,2146.0,684.0,24.0,0.0,0.0,0.0,50.0,8.0,4.0,4.0,0.0,6.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401120,6260687.0,29000.0,1074871.0,4331.0,149.0,2.0,1000.0,0.0,0.0,3182.0,1063.0,24.0,0.0,0.0,0.0,58.0,31.0,5.0,5.0,0.0,3.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,17.0,2.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
401121,6312170.0,11000.0,1812622.0,9580.0,149.0,2.0,2005.0,0.0,0.0,4579.0,1699.0,0.0,0.0,0.0,0.0,

In [18]:
train_df_filled[num_missing].isna().sum()

SalePrice                   0
auctioneerID                0
MachineHoursCurrentMeter    0
dtype: int64

In [19]:
valid_df_filled[num_missing].isna().sum()

SalePrice                   0
auctioneerID                0
MachineHoursCurrentMeter    0
dtype: int64

In [0]:
# separating features and labels
X_train_filled,y_train_filled = train_df_filled.drop(['SalePrice'],axis=1),train_df_filled.SalePrice 
X_valid_filled,y_valid_filled = valid_df_filled.drop(['SalePrice'],axis=1),valid_df_filled.SalePrice

X_train,y_train = train_df.drop(['SalePrice'],axis=1),train_df_filled.SalePrice
X_valid,y_valid = valid_df.drop(['SalePrice'],axis=1),valid_df_filled.SalePrice

In [0]:
# array to store diffrent hyperparameters of RandomForestClassifier and respective scores
models=[]

In [0]:
# function to evaluate diffrent metrics 
def evaluate(model, X_valid, y_valid):
    y_pred = model.predict(X_valid)
    scores = {
        'R2': model.score(X_valid, y_valid),
        'MAE': mean_absolute_error(y_valid, y_pred),
        'RMLE': np.sqrt(mean_squared_log_error(y_valid, y_pred))}
    return(scores)

In [0]:
# function to compile hyperparameters and scores of a model
def store_result(model,params, X,y):
    y_pred = model.predict(X)
    scores = {
        'R2': model.score(X, y),
        'MAE': mean_absolute_error(y, y_pred),
        'RMLE': np.sqrt(mean_squared_log_error(y, y_pred))}
    model = {
        'scores': scores,
        'params': params}
    if model not in models:
        models.append(model)
    return(model)

In [0]:
rfr = RandomForestRegressor(n_jobs=-1,n_estimators=100,max_depth=4)

In [25]:
rfr.fit(X_train_filled,y_train_filled) # fitting fileed training data

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=4, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [29]:
evaluate(rfr,X_valid_filled,y_valid_filled)

{'MAE': 11743.591695547851,
 'R2': 0.5750981549867145,
 'RMLE': 0.46066602400619483}

In [0]:
param_grid={
    'randomforestregressor__n_estimators':np.arange(90,150,10),
    'randomforestregressor__max_depth':[None],
    'randomforestregressor__min_samples_split':np.arange(2,20,1),
    'randomforestregressor__min_samples_leaf':np.arange(1,15,1)
} 

In [0]:
num_imputer = SimpleImputer(strategy='median')
transformer = ColumnTransformer(transformers=[('num_missing',num_imputer,X_train.columns)])
rfr = RandomForestRegressor(random_state=23,max_samples=10000,n_jobs=-1) # tuning using only 10000 samples

In [0]:
from sklearn.pipeline import make_pipeline
rfr_pipeline= make_pipeline(transformer,rfr)

In [0]:
scorer_func= lambda y_true,y_pred: np.sqrt(mean_squared_log_error(y_true,y_pred)) # custom scorer function of rmsle

In [0]:
scorer = make_scorer(scorer_func,greater_is_better=False)

In [0]:
rfr_random = RandomizedSearchCV(estimator=rfr_pipeline,
                                param_distributions=param_grid,
                                cv=5,
                                n_jobs=-1,
                                n_iter=2,
                                scoring=scorer)

In [40]:
%%time
rfr_random.fit(X_train,y_train) #

CPU times: user 45.3 s, sys: 752 ms, total: 46.1 s
Wall time: 4min 41s


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('columntransformer',
                                              ColumnTransformer(n_jobs=None,
                                                                remainder='drop',
                                                                sparse_threshold=0.3,
                                                                transformer_weights=None,
                                                                transformers=[('num_missing',
                                                                               SimpleImputer(add_indicator=False,
                                                                                             copy=True,
                                                                                             fill_value=None,
                                                                                   

In [41]:
#fitting entire training data to the best estimator
rfr = rfr_random.best_estimator_.set_params(randomforestregressor__max_samples = None);
rfr.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num_missing',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='median',
                                                                verbose=0),
                                                  Index(['SalesID', 'MachineID', 'ModelID', 'datasource', 'auctioneerID',
       'Yea...
                 RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,


In [42]:
store_result(rfr,rfr_random.best_params_,X_valid,y_valid)

{'params': {'randomforestregressor__max_depth': None,
  'randomforestregressor__min_samples_leaf': 2,
  'randomforestregressor__min_samples_split': 11,
  'randomforestregressor__n_estimators': 130},
 'scores': {'MAE': 6041.3989151214955,
  'R2': 0.8748941854973731,
  'RMLE': 0.2498230406910499}}

In [0]:
transformer = ColumnTransformer(transformers=[('num_missing',num_imputer,X_train.columns)])
rfr = RandomForestRegressor(random_state=23,max_samples=4000,n_jobs=-1)
rfr_pipeline= make_pipeline(transformer,rfr)

In [0]:
param_grid={
    'randomforestregressor__n_estimators':np.arange(110,121,1),
    'randomforestregressor__max_depth':[None],
    'randomforestregressor__min_samples_split':[19],
    'randomforestregressor__min_samples_leaf':[7]
}

In [0]:
rfr_grid = GridSearchCV(estimator=rfr_pipeline,
                                param_grid=param_grid,
                                cv=5,
                                n_jobs=-1,
                                scoring=scorer)

In [46]:
rfr_grid.fit(X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('columntransformer',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('num_missing',
                                                                         SimpleImputer(add_indicator=False,
                                                                                       copy=True,
                                                                                       fill_value=None,
                                                                                       missing_values=nan,
                                          

In [47]:

rfr = rfr_grid.best_estimator_.set_params(randomforestregressor__max_samples = None);
rfr.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num_missing',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='median',
                                                                verbose=0),
                                                  Index(['SalesID', 'MachineID', 'ModelID', 'datasource', 'auctioneerID',
       'Yea...
                 RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,


In [48]:
store_result(rfr,rfr_grid.best_params_,X_valid,y_valid)

{'params': {'randomforestregressor__max_depth': None,
  'randomforestregressor__min_samples_leaf': 7,
  'randomforestregressor__min_samples_split': 19,
  'randomforestregressor__n_estimators': 119},
 'scores': {'MAE': 6070.621518230695,
  'R2': 0.871926457892915,
  'RMLE': 0.24897174385709098}}

In [49]:
models

[{'params': {'randomforestregressor__max_depth': None,
   'randomforestregressor__min_samples_leaf': 2,
   'randomforestregressor__min_samples_split': 11,
   'randomforestregressor__n_estimators': 130},
  'scores': {'MAE': 6041.3989151214955,
   'R2': 0.8748941854973731,
   'RMLE': 0.2498230406910499}},
 {'params': {'randomforestregressor__max_depth': None,
   'randomforestregressor__min_samples_leaf': 7,
   'randomforestregressor__min_samples_split': 19,
   'randomforestregressor__n_estimators': 119},
  'scores': {'MAE': 6070.621518230695,
   'R2': 0.871926457892915,
   'RMLE': 0.24897174385709098}}]